In [ ]:
### import the following libraries
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
from PIL import Image
import os
import warnings
import tifffile as tiff

warnings.filterwarnings('ignore')

In [ ]:
### directory & filepaths
data_dir = '/home/shamini/data1/data_orig/data/spatial/xenium/10xGenomics/cell_seg_brain_cancer/'
out = '/home/shamini/data/projects/spatial_workshop/out/module6/'
os.makedirs(out+'script05_figures/', exist_ok=True)

In [ ]:
transcripts = 'transcripts.parquet'
image = 'morphology_focus_0000.ome.tif'

In [ ]:
df_transcript = pd.read_parquet(data_dir+'out/'+transcripts)
iF = tiff.imread(data_dir+'out/morphology_focus/'+image)
#he = tiff.imread(data_dir+he)

In [ ]:
scale = 0.2125

In [ ]:
df_transcript[['x_location']] = df_transcript[['x_location']]/scale
df_transcript[['y_location']] = df_transcript[['y_location']]/scale

In [ ]:
df_transcript

In [ ]:
xmin_cut = int(10000/scale)
xmax_cut = int(11000/scale)

ymin_cut = int(6000/scale)
ymax_cut = int(7000/scale)

print(xmin_cut, xmax_cut, ymin_cut, ymax_cut)

In [ ]:
transcripts_subset = df_transcript[((df_transcript['x_location'] > xmin_cut) & (df_transcript['x_location'] < xmax_cut)) & ((df_transcript['y_location'] > ymin_cut) & (df_transcript['y_location'] < ymax_cut))]

In [ ]:
sns.scatterplot(data=transcripts_subset, x='x_location', y='y_location', s=.1)

In [ ]:
min_x = int(transcripts_subset['x_location'].min())
max_x = int(transcripts_subset['x_location'].max())

min_y = int(transcripts_subset['y_location'].min())
max_y = int(transcripts_subset['y_location'].max())

In [ ]:
print(min_x, max_x, min_y, max_y)

In [ ]:
### reset the coordinates to 0
transcripts_subset['x_location'] = transcripts_subset['x_location'] - min_x
transcripts_subset['y_location'] = transcripts_subset['y_location'] - min_y

transcripts_subset.to_csv(out+'transcripts_subset_all_genes.csv')

In [ ]:
### crop the image
fig, ax = plt.subplots(2, 2, figsize=(16, 16))
iF_crop = iF[:, min_y:max_y, min_x:max_x]
ax = ax.flatten()

for channel in range(4):
    ax[channel].imshow(iF_crop[channel], cmap='gray')
    ax[channel].axis('off')
    ax[channel].set_title(f'Channel {channel}') 

In [ ]:
### save the cropped image
tiff.imsave(out+'script06_objects/cropped_image.tif', iF_crop)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 8))
transcripts_subset_sample = transcripts_subset.sample(frac=0.3, random_state=1)

axes[0].imshow(iF_crop[1,:,:], cmap='magma')
ax = sns.scatterplot(data=transcripts_subset_sample, x='x_location', y='y_location', s=0.3, ax=axes[0], alpha=0.5, color='white')
axes[0].set_title('Original Image')
axes[0].axis('off')

axes[1].imshow(iF_crop[1,:,:], cmap='magma')
ax = sns.scatterplot(data=transcripts_subset_sample, x='x_location', y='y_location', s=0.9, ax=axes[1], alpha=0.75, color='white')
ax.set_xlim(3000, 3500)
ax.set_ylim(3000, 3500)

In [ ]:
### import cell boundaries and nuclear boundaries files
cell_boundaries = pd.read_parquet(data_dir+'out/cell_boundaries.parquet')
nuclear_boundaries = pd.read_parquet(data_dir+'out/nucleus_boundaries.parquet')

In [ ]:
cell_boundaries = cell_boundaries[cell_boundaries['cell_id'].isin(df_transcript['cell_id'])]
cell_boundaries['vertex_x'] = (cell_boundaries['vertex_x']/0.2125)
cell_boundaries['vertex_y'] = (cell_boundaries['vertex_y']/0.2125)

nuclear_boundaries = nuclear_boundaries[nuclear_boundaries['cell_id'].isin(df_transcript['cell_id'])]
nuclear_boundaries['vertex_x'] = (nuclear_boundaries['vertex_x']/0.2125)
nuclear_boundaries['vertex_y'] = (nuclear_boundaries['vertex_y']/0.2125)

In [ ]:
### shift the coordinates to match the cropped image
cell_boundaries['vertex_x'] = cell_boundaries['vertex_x'] - min_x
cell_boundaries['vertex_y'] = cell_boundaries['vertex_y'] - min_y

nuclear_boundaries['vertex_x'] = nuclear_boundaries['vertex_x'] - min_x
nuclear_boundaries['vertex_y'] = nuclear_boundaries['vertex_y'] - min_y

In [ ]:
cell_boundaries.to_parquet(out+'cell_boundaries_subset.parquet')
nuclear_boundaries.to_parquet(out+'nuclear_boundaries_subset.parquet')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Check the data's shape and maximum value
print("Shape of iF_crop:", iF_crop.shape)
for i in range(4):
    print(f"Channel {i} max: {np.max(iF_crop[i])}, min: {np.min(iF_crop[i])}")

# Define colors, with DAPI visualized in blue
colors = [
    np.array([0, 0, 1]),  # Blue for DAPI
    np.array([1, 0, 0]),  # Red
    np.array([0, 1, 0]),  # Green
    np.array([1, 1, 0])   # Yellow for the fourth channel
]

# Create a composite image with 3 channels (RGB)
composite_image = np.zeros((iF_crop.shape[1], iF_crop.shape[2], 3), dtype=np.float32)

# Assign each channel to a color in RGB, using a more conservative normalization approach
for i in range(4):
    channel_data = iF_crop[i, :, :]
    if np.max(channel_data) > 0:  # Avoid division by zero and unnecessary normalization
        normalized_data = channel_data / np.max(channel_data)
        composite_image += normalized_data[:, :, np.newaxis] * colors[i]

#composite_image += channel_data[:, :, np.newaxis] * colors[i]


# Ensure the image is normalized to the maximum of the composite_image
#if np.max(composite_image) > 0:
#    composite_image /= np.max(composite_image)


In [ ]:

fig, axes = plt.subplots()
axes.imshow(composite_image)



In [ ]:
### save the composite image as tiff
tiff.imsave(out+'script05_figures/composite_image.tif', composite_image)

In [ ]:
#### subset adata to the cropped image

adata = sc.read_h5ad(out+'adata.h5ad')
print(f'Adata before cropping:\n {adata}\n')
adata_crop = adata[adata.obs.index.isin(transcripts_subset['cell_id']),:]
print(f'Adata after cropping:\n {adata_crop}')

In [ ]:
genes = ['PTPRC', 'ANXA1', 'STMN1']

adata_crop = adata_crop[:,genes]
adata_crop.write_h5ad(out+'adata_crop.h5ad')    

In [ ]:
transcripts_subset = transcripts_subset[transcripts_subset['feature_name'].isin(genes)]
transcripts_subset.to_csv(out+'transcripts_subset_genes.csv', index=False)

In [ ]:
cell_boundaries = cell_boundaries[cell_boundaries['cell_id'].isin(transcripts_subset['cell_id'])]
nuclear_boundaries = nuclear_boundaries[nuclear_boundaries['cell_id'].isin(transcripts_subset['cell_id'])]

cell_boundaries.to_csv(out+'cell_boundaries_subset.csv', index=False)
nuclear_boundaries.to_csv(out+'nuclear_boundaries_subset.csv', index=False) 

In [ ]:
transcripts_subset